# 3.Twitter Sentiment Analysis: Training The data With Augmentation

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import nlpaug.augmenter.word as naw
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
sns.set_style('darkgrid')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline
sns.set_style("whitegrid")

In [2]:
data = pd.read_csv('train.csv')
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data.drop("id",axis=1,inplace= True)

In [5]:
# removes pattern in the input text
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "", input_txt)
    return input_txt

In [6]:
# remove twitter handles (@user)
data['clean_tweet'] = np.vectorize(remove_pattern)(data['tweet'], "@[\w]*")

In [7]:
def clean_documents(document):
  stop = stopwords.words('english')
  tokens = word_tokenize(document.lower())

  word_tokens = [token for token in tokens if token.isalpha()]
  clean_tokens = [token for token in word_tokens if token not in stop]

  lemmatizer = WordNetLemmatizer()
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in clean_tokens]

  return " ".join(lemmatized_tokens)

In [8]:
data["clean_tweet"] = data["clean_tweet"].apply(clean_documents)

In [9]:
X=data.clean_tweet
y=data.label

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)

In [12]:
X_train.head()

18168    russian default position faced accusation dopi...
15286    u looked one hairline already receiving tf hai...
4964         wow finally see southcitymall felling waiting
5373                   always bereft finish something like
24201       hispanic amp feel like stomping listen retweet
Name: clean_tweet, dtype: object

In [13]:
y_train.head()

18168    0
15286    0
4964     0
5373     0
24201    1
Name: label, dtype: int64

In [14]:
data= pd.concat([X_train,y_train], axis=1)
data.head()

,clean_tweet,label
18168,russian default position faced accusation dopi...,0
15286,u looked one hairline already receiving tf hai...,0
4964,wow finally see southcitymall felling waiting,0
5373,always bereft finish something like,0
24201,hispanic amp feel like stomping listen retweet,1


In [15]:
 #nplaug
import nltk
nltk.download('averaged_perceptron_tagger')
data_resampled_nlpaug = data.copy()

aug_texts = []
minority_data = data_resampled_nlpaug[data_resampled_nlpaug["label"] == 1]
aug = naw.SynonymAug(aug_src='wordnet')

texts = minority_data['clean_tweet'].tolist()

for text in texts:
    augmented_texts = aug.augment(text, n=12)
    
    for augmented in augmented_texts:
        aug_texts.append(augmented)

print(len(aug_texts))

temp = pd.DataFrame({
    'clean_tweet': aug_texts
})
        
temp["label"] = 1
        
data_resampled_nlpaug = pd.concat([data_resampled_nlpaug, temp], axis=0)
data_resampled_nlpaug = data_resampled_nlpaug.reset_index()
data_resampled_nlpaug = data_resampled_nlpaug.drop(columns=['index'])
del temp, minority_data

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\abhim\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


19068


In [16]:
data_resampled_nlpaug

,clean_tweet,label
0,russian default position faced accusation dopi...,0
1,u looked one hairline already receiving tf hai...,0
2,wow finally see southcitymall felling waiting,0
3,always bereft finish something like,0
4,hispanic amp feel like stomping listen retweet,1
...,...,...
41436,come a documented liar lost home calling nazi ...,1
41437,come a documented liar lost base calling natio...,1
41438,come up ampere documented liar lost base calli...,1
41439,make out a documented liar lost base calling n...,1


In [17]:
df=data_resampled_nlpaug

In [18]:
df.isnull().sum()

clean_tweet    0
label          0
dtype: int64

In [19]:
X_train=df.clean_tweet
y_train=df.label

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
# CountVectorizer
cv = CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [22]:
# import Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
rfc= RandomForestClassifier()
rfc.fit(X_train_cv, y_train) 
y_pred = rfc.predict(X_test_cv)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.08      0.16      8940
           1       0.07      1.00      0.14       649

    accuracy                           0.15      9589
   macro avg       0.54      0.54      0.15      9589
weighted avg       0.94      0.15      0.15      9589



In [23]:
# TfidfVectorizer
tf = TfidfVectorizer(stop_words= 'english', ngram_range= (1,3), lowercase= True, max_features= 5000)
X_train_tf = tf.fit_transform(X_train)
X_test_tf = tf.transform(X_test)


In [24]:
rfc= RandomForestClassifier()
rfc.fit(X_train_tf, y_train) 
y_pred = rfc.predict(X_test_tf)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      8940
           1       0.58      0.68      0.63       649

    accuracy                           0.95      9589
   macro avg       0.78      0.82      0.80      9589
weighted avg       0.95      0.95      0.95      9589



In [25]:
from sklearn.svm import LinearSVC
# HARD MARGIN
lin=LinearSVC(random_state=42)
lin.fit(X_train_tf, y_train)
y_pred = lin.predict(X_test_tf)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96      8940
           1       0.51      0.74      0.60       649

    accuracy                           0.93      9589
   macro avg       0.74      0.84      0.78      9589
weighted avg       0.95      0.93      0.94      9589



In [26]:
# NN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, SimpleRNN, Bidirectional, LSTM, GRU 

In [27]:
# Processing text

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [28]:
vocabulary = tokenizer.index_word
vocabulary

{1: 'amp',
 2: 'love',
 3: 'day',
 4: 'u',
 5: 'like',
 6: 'trump',
 7: 'libtard',
 8: 'black',
 9: 'white',
 10: 'people',
 11: 'happy',
 12: 'new',
 13: 'get',
 14: 'time',
 15: 'woman',
 16: 'life',
 17: 'good',
 18: 'make',
 19: 'today',
 20: 'racist',
 21: 'one',
 22: 'go',
 23: 'allahsoil',
 24: 'feel',
 25: 'want',
 26: 'see',
 27: 'girl',
 28: 'politics',
 29: 'need',
 30: 'ca',
 31: 'thankful',
 32: 'say',
 33: 'retweet',
 34: 'father',
 35: 'positive',
 36: 'obama',
 37: 'liberal',
 38: 'hate',
 39: 'listen',
 40: 'sjw',
 41: 'year',
 42: 'might',
 43: 'take',
 44: 'think',
 45: 'smile',
 46: 'would',
 47: 'bihday',
 48: 'work',
 49: 'got',
 50: 'america',
 51: 'video',
 52: 'great',
 53: 'friend',
 54: 'via',
 55: 'look',
 56: 'thing',
 57: 'way',
 58: 'thanks',
 59: 'stop',
 60: 'really',
 61: 'never',
 62: 'right',
 63: 'family',
 64: 'world',
 65: 'know',
 66: 'back',
 67: 'show',
 68: 'week',
 69: 'man',
 70: 'call',
 71: 'guy',
 72: 'best',
 73: 'going',
 74: 'much',
 7

In [29]:
vocab_len = len(vocabulary)
vocab_len

29468

In [30]:
train_sequence = tokenizer.texts_to_sequences(X_train)

In [31]:
doc_len = []
for doc in train_sequence:
    doc_len.append(len(doc))

In [32]:
max(doc_len)

37

In [33]:
np.quantile(doc_len, 0.9999)

21.0

In [34]:
max_len = 21

In [35]:
train_sequence_matrix = sequence.pad_sequences(train_sequence, maxlen= max_len)

In [36]:
# testing
test_sequence = tokenizer.texts_to_sequences(X_test)
test_sequence_matrix = sequence.pad_sequences(test_sequence, maxlen= max_len)

In [37]:
# model

model = Sequential()
model.add(Embedding(input_dim=vocab_len+1,output_dim=100,input_length=max_len,mask_zero=True))
model.add(Flatten())
model.add(Dense(64,activation="tanh"))
model.add(Dense(64,activation="tanh"))
model.add(Dense(1,activation="sigmoid"))

In [38]:
model.compile(optimizer="adam", loss="binary_crossentropy")

In [39]:
model.fit(train_sequence_matrix,y_train, batch_size=32,epochs=10)

Epoch 1/10
1296/1296 [==============================] - 44s 34ms/step - loss: 0.1104
Epoch 2/10
1296/1296 [==============================] - 40s 31ms/step - loss: 0.0072
Epoch 3/10
1296/1296 [==============================] - 40s 31ms/step - loss: 0.0035
Epoch 4/10
1296/1296 [==============================] - 40s 31ms/step - loss: 0.0028
Epoch 5/10
1296/1296 [==============================] - 40s 31ms/step - loss: 0.0026
Epoch 6/10
1296/1296 [==============================] - 40s 31ms/step - loss: 0.0018
Epoch 7/10
1296/1296 [==============================] - 40s 31ms/step - loss: 0.0015
Epoch 8/10
1296/1296 [==============================] - 42s 33ms/step - loss: 0.0017
Epoch 9/10
1296/1296 [==============================] - 39s 30ms/step - loss: 0.0013
Epoch 10/10
1296/1296 [==============================] - 40s 31ms/step - loss: 0.0012


In [40]:
y_pred = model.predict(test_sequence_matrix)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      8940
           1       0.57      0.58      0.57       649

    accuracy                           0.94      9589
   macro avg       0.77      0.77      0.77      9589
weighted avg       0.94      0.94      0.94      9589



In [41]:
# Bi-directional
model = Sequential()
model.add(Embedding(input_dim=vocab_len+1,output_dim=100,input_length=max_len,mask_zero=True))
model.add(Bidirectional(SimpleRNN(32, activation="tanh")))
model.add(Dense(64,activation="tanh"))
model.add(Dense(64,activation="tanh"))
model.add(Dense(1,activation="sigmoid"))

In [42]:
model.compile(optimizer="adam", loss="binary_crossentropy")
model.fit(train_sequence_matrix,y_train, batch_size=32,epochs=10)

Epoch 1/10
1296/1296 [==============================] - 62s 46ms/step - loss: 0.0999
Epoch 2/10
1296/1296 [==============================] - 83s 64ms/step - loss: 0.0056 0s - loss
Epoch 3/10
1296/1296 [==============================] - 83s 64ms/step - loss: 0.0042 0s - loss: 0.00
Epoch 4/10
1296/1296 [==============================] - 86s 66ms/step - loss: 0.0031
Epoch 5/10
1296/1296 [==============================] - 89s 69ms/step - loss: 0.0040 0s - loss: 0.004
Epoch 6/10
1296/1296 [==============================] - 85s 66ms/step - loss: 0.0021
Epoch 7/10
1296/1296 [==============================] - 94s 72ms/step - loss: 9.2554e-04
Epoch 8/10
1296/1296 [==============================] - 93s 72ms/step - loss: 0.0011
Epoch 9/10
1296/1296 [==============================] - 88s 68ms/step - loss: 0.0012
Epoch 10/10
1296/1296 [==============================] - 92s 71ms/step - loss: 0.0025- ETA: 0s - loss: 0.0


In [43]:
y_pred = model.predict(test_sequence_matrix)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      8940
           1       0.63      0.58      0.61       649

    accuracy                           0.95      9589
   macro avg       0.80      0.78      0.79      9589
weighted avg       0.95      0.95      0.95      9589



In [44]:
# LSTM
model = Sequential()
model.add(Embedding(input_dim=vocab_len+1,output_dim=100,input_length=max_len,mask_zero=True))
model.add(LSTM(128, activation="tanh"))
model.add(Dense(128,activation="tanh"))
model.add(Dense(64,activation="tanh"))
model.add(Dense(64,activation="tanh"))
model.add(Dense(64,activation="tanh"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy")
model.fit(train_sequence_matrix,y_train, batch_size=32,epochs=10)

Epoch 1/10
1296/1296 [==============================] - 144s 98ms/step - loss: 0.1213
Epoch 2/10
1296/1296 [==============================] - 133s 102ms/step - loss: 0.0194s - - ETA: 0s - loss
Epoch 3/10
1296/1296 [==============================] - 133s 103ms/step - loss: 0.0093s - l
Epoch 4/10
1296/1296 [==============================] - 125s 96ms/step - loss: 0.00681
Epoch 5/10
1296/1296 [==============================] - 122s 94ms/step - loss: 0.0043
Epoch 6/10
1296/1296 [==============================] - 173s 133ms/step - loss: 0.0047
Epoch 7/10
1296/1296 [==============================] - 169s 130ms/step - loss: 0.0032
Epoch 8/10
1296/1296 [==============================] - 175s 135ms/step - loss: 0.0034
Epoch 9/10
1296/1296 [==============================] - 174s 134ms/step - loss: 0.0029
Epoch 10/10
1296/1296 [==============================] - 167s 129ms/step - loss: 0.0021


In [45]:
y_pred = model.predict(test_sequence_matrix)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      8940
           1       0.60      0.61      0.61       649

    accuracy                           0.95      9589
   macro avg       0.79      0.79      0.79      9589
weighted avg       0.95      0.95      0.95      9589



In [46]:
# multilayer LSTM
model = Sequential()
model.add(Embedding(input_dim=vocab_len+1,output_dim=100,input_length=max_len,mask_zero=True))
model.add(LSTM(32, activation="tanh", return_sequences=True))
model.add(LSTM(32, activation="tanh"))
model.add(Dense(64,activation="tanh"))
model.add(Dense(64,activation="tanh"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy")
model.fit(train_sequence_matrix,y_train, batch_size=32,epochs=10)

Epoch 1/10
1296/1296 [==============================] - 217s 131ms/step - loss: 0.1210
Epoch 2/10
1296/1296 [==============================] - 94s 72ms/step - loss: 0.0175
Epoch 3/10
1296/1296 [==============================] - 89s 68ms/step - loss: 0.0095
Epoch 4/10
1296/1296 [==============================] - 100s 77ms/step - loss: 0.0045
Epoch 5/10
1296/1296 [==============================] - 97s 75ms/step - loss: 0.0035
Epoch 6/10
1296/1296 [==============================] - 101s 78ms/step - loss: 0.0027
Epoch 7/10
1296/1296 [==============================] - 94s 72ms/step - loss: 0.0029
Epoch 8/10
1296/1296 [==============================] - 98s 76ms/step - loss: 0.0029
Epoch 9/10
1296/1296 [==============================] - 90s 70ms/step - loss: 0.0022
Epoch 10/10
1296/1296 [==============================] - 89s 69ms/step - loss: 0.0013


In [47]:
y_pred = model.predict(test_sequence_matrix)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      8940
           1       0.70      0.56      0.62       649

    accuracy                           0.95      9589
   macro avg       0.83      0.77      0.80      9589
weighted avg       0.95      0.95      0.95      9589



In [48]:
# Bidirection LSTM
model = Sequential()
model.add(Embedding(input_dim=vocab_len+1,output_dim=100,input_length=max_len,mask_zero=True))
model.add(Bidirectional(LSTM(32, activation="tanh")))
model.add(Dense(64,activation="tanh"))
model.add(Dense(64,activation="tanh"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy")
model.fit(train_sequence_matrix,y_train, batch_size=32,epochs=10)

Epoch 1/10
1296/1296 [==============================] - 111s 58ms/step - loss: 0.1123
Epoch 2/10
1296/1296 [==============================] - 73s 56ms/step - loss: 0.0121
Epoch 3/10
1296/1296 [==============================] - 78s 60ms/step - loss: 0.0050
Epoch 4/10
1296/1296 [==============================] - 76s 59ms/step - loss: 0.0039
Epoch 5/10
1296/1296 [==============================] - 75s 58ms/step - loss: 0.0031
Epoch 6/10
1296/1296 [==============================] - 81s 62ms/step - loss: 0.0023
Epoch 7/10
1296/1296 [==============================] - 78s 60ms/step - loss: 0.0013
Epoch 8/10
1296/1296 [==============================] - 78s 60ms/step - loss: 0.0014
Epoch 9/10
1296/1296 [==============================] - 74s 57ms/step - loss: 0.0015
Epoch 10/10
1296/1296 [==============================] - 75s 58ms/step - loss: 0.0013


In [49]:
y_pred = model.predict(test_sequence_matrix)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      8940
           1       0.61      0.63      0.62       649

    accuracy                           0.95      9589
   macro avg       0.79      0.80      0.80      9589
weighted avg       0.95      0.95      0.95      9589



In [50]:
# GRU Bidirectional
model = Sequential()
model.add(Embedding(input_dim=vocab_len+1,output_dim=100,input_length=max_len,mask_zero=True))
model.add(Bidirectional(GRU(128, activation="tanh")))
model.add(Dense(128,activation="tanh"))
model.add(Dense(64,activation="tanh"))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy")
model.fit(train_sequence_matrix,y_train, batch_size=64,epochs=50)

Epoch 1/50
648/648 [==============================] - 77s 80ms/step - loss: 0.1173
Epoch 2/50
648/648 [==============================] - 51s 79ms/step - loss: 0.0131
Epoch 3/50
648/648 [==============================] - 52s 80ms/step - loss: 0.0049
Epoch 4/50
648/648 [==============================] - 52s 80ms/step - loss: 0.0029
Epoch 5/50
648/648 [==============================] - 53s 82ms/step - loss: 0.0027
Epoch 6/50
648/648 [==============================] - 54s 83ms/step - loss: 0.0050
Epoch 7/50
648/648 [==============================] - 53s 82ms/step - loss: 0.0035
Epoch 8/50
648/648 [==============================] - 54s 83ms/step - loss: 0.0014
Epoch 9/50
648/648 [==============================] - 54s 83ms/step - loss: 0.0013
Epoch 10/50
648/648 [==============================] - 58s 90ms/step - loss: 0.0015
Epoch 11/50
648/648 [==============================] - 58s 90ms/step - loss: 0.0011
Epoch 12/50
648/648 [==============================] - 59s 92ms/step - loss: 8.5170e-

In [51]:
y_pred = model.predict(test_sequence_matrix)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      8940
           1       0.67      0.57      0.62       649

    accuracy                           0.95      9589
   macro avg       0.82      0.78      0.80      9589
weighted avg       0.95      0.95      0.95      9589

